In [2]:
import torch
from IPython.display import Audio, display
import numpy as np
import librosa as li
import soundfile as sf
import random
import matplotlib.pyplot as plt
import math

#IPython.display.Audio("my_audio_file.mp3")

In [6]:
def generate_random(model_dir):
    """
    This function uses a specified torchscript model to generate 10 seconds of audio
    Params:
        model_dir: file path to the torchscript file to use as the model in string format
        output_dir: file path to where you would like the audio saved too
    """
    model = torch.jit.load(model_dir).eval()

    sr = 44100
    z = torch.rand(1, 8, 862)
    
    x = model.decode(z).detach().numpy().reshape(-1)

    return x, sr

In [ ]:
maestro_ones_ts = "/opt/research/datasets/maestrowithones/maestroWithOnes.ts"

x_hat1, sr1 = generate_random(maestro_ones_ts)

sf.write("random.wav", x_hat, sr)
display(Audio(x_hat1, rate=sr1))

In [38]:
def generate_shapes(model_dir, shape=None):
    """ 
        Valid shapes:
            - "circle"
    """
    sr = 44100
    model = torch.jit.load(model_dir).eval()
    
    z = torch.rand(1, 8, 862)
    z = torch.zeros_like(z)

    for i in range(len(z)):
        for j in range(len(z[i])):
            for k in range(len(z[i][j])):
                val = math.sqrt(k**2 + j**2 + i**2)
                z[i][j][k] = val

    x = model.decode(z).detach().numpy().reshape(-1)

    return x, sr


In [ ]:
x_hat, sr = generate_shapes(maestro_ones_ts)

display(Audio(x_hat, rate=sr))

In [ ]:
new_guy =  np.abs(x_hat - x_hat1)
display(Audio(new_guy, rate=sr))

In [18]:
def generate_from_audio(model_dir, audio_loc):
    
    model = torch.jit.load(model_dir).eval()
    x, sr = li.load(audio_loc, sr=44100)
    x = torch.from_numpy(x).reshape(1,1,-1)
    z = model.encode(x)
    x_hat = model.decode(z).detach().numpy().reshape(-1)
    sr = 44100

    return x_hat, sr

In [ ]:
x, sr = generate_from_audio(maestro_ones_ts, "/opt/research/garnerviolin/IV. Double Presto.mp3")
sf.write("violin.wav", x_hat, sr)
display(Audio(x, rate=sr))

In [ ]:
maestro_ones_ts = "/opt/research/datasets/maestrowithones/maestroWithOnes.ts"

model = torch.jit.load(maestro_ones_ts).eval()

sr = 44100
# Dimensions should be 1, 8, 862
arr1 = np.linspace(0.001, 1, 141, endpoint=False)
arr1 = arr1.astype(np.float32, order='K', casting='unsafe', subok=True, copy=True)
print(arr1.dtype)
arr = []
for i in range(8):
    to_append = arr1
    if i%2 == 0:
        to_append = to_append[::-1]
    arr.append([to_append])

arr = np.array(arr)

z = torch.from_numpy(arr)

print(z)
x = model.decode(z).detach().numpy().reshape(-1)

display(Audio(x, rate=sr))

In [ ]:
maestro_ones_ts = "/opt/research/datasets/maestrowithones/maestroWithOnes.ts"
maestro_no_ones_ts = "/opt/research/AudioSteno/datasets/maestro/train/maestro_train_original_e18d54798e/version_0/checkpoints/maestro_train_original_e18d54798e.ts"
model = torch.jit.load(maestro_ones_ts).eval()

sr = 44100
# Dimensions should be 1, 8, 862
arr1 = np.linspace(0.001, 1, 60, endpoint=False)
print(arr1.size)
arr1 = arr1.astype(np.float32, order='K', casting='unsafe', subok=True, copy=True)
print(arr1.dtype)
arr = []
for i in range(128):
    to_append = arr1
    if i%2 == 0:
        to_append = to_append[::-1]
    arr.append([to_append])

arr = np.array(arr)

z = torch.from_numpy(arr)

print(z)
x = model.decode(z).detach().numpy().reshape(-1)

display(Audio(x, rate=sr))

In [20]:
def create_shape(pts, shape): #Assumes two dimensions
    ## Now create a curve in d dimensions and put it back into dim space using the found basis
    X = shape # 50 points in d dimensions
    Y = U.dot(X) # Here are the same points on the plane in dim dimensions
    return Y
def rectangle_old(l, w, center, pts):
    if pts%4 != 0:
        pts += -pts%4
        print("Constructing rectangle with {} points".format(pts))
    ret = np.zeros((2, pts))
    centerX = center[0]
    startX = centerX-(w/2)
    side1X = np.linspace(startX, startX+w, pts//4)
    side2X = np.linspace(startX+w, startX, pts//4)
    ret[0][0:pts//4] = side1X
    ret[0][pts//4:(2*pts)//4] = startX+w
    ret[0][(2*pts)//4:(3*pts)//4] = side2X
    ret[0][(3*pts)//4::] = startX
    
    centerY = center[1]
    startY = centerY-(l/2)
    side1Y = np.linspace(startY, startY+l, pts//4)
    side2Y = np.linspace(startY+l, startY, pts//4)
    ret[1][0:pts//4] = startY
    ret[1][pts//4:(2*pts)//4] = side1Y
    ret[1][(2*pts)//4:(3*pts)//4] = startY+l
    ret[1][(3*pts)//4::] = side2Y
    return ret


def ellipse(xr, yr, center, pts):
    ell = np.zeros((2, pts))
    theta = np.linspace(0, 2*np.pi, pts)

    x = (center[0] + xr)*np.cos(theta)
    y = (center[1] + yr)*np.sin(theta)
    ell[0] = x
    ell[1] = y
    return ell

In [2]:
def eq_triangle(s, r, center, pts):
    if pts%3 != 0:
        pts += -pts%3
        print("Constructing rectangle with {} points".format(pts))
    tri = np.zeros((2, pts))
    h_c = np.sqrt(r**2-(s/2)**2)
    startX = center[0] - (s/2)
    startY = center[1] - h_c
    side1X = np.linspace(startX, startX+s, pts//3)
    side2X = np.linspace(startX+s, startX+(s/2), pts//3)
    side3X = np.linspace(startX+(s/2), startX, pts//3)
    tri[0][0:pts//3] = side1X
    tri[0][pts//3:(2*pts)//3] = side2X
    tri[0][(2*pts)//3::] = side3X
    
    side1Y = np.linspace(startY, startY, pts//3)
    side2Y = np.linspace(startY, startY+s, pts//3)
    side3Y = np.linspace(startY+s, startY, pts//3)
    tri[1][0:pts//3] = side1Y
    tri[1][pts//3:(2*pts)//3] = side2Y
    tri[1][(2*pts)//3::] = side3Y
    return tri

In [3]:
import numpy as np
def create_shape(R, dim, d):
    """
    Create a shape in d dimensions, that then gets sent back up to the latent space dimension.

    Parameters:
    pts : int
        Number of points that composes the shape.
    shape : function
        The function necessary to make a certain shape in the 2-dimensional space.
    dim : int
        Original dimension of space
    d : int
        The dimension the orthonormal basis is created in

    Returns:
    Y : np.array(dim, pts)
        Returns the array that is sent back up into the latent space dimension.
    """
    ## Step 1: Find an orthonormal basis for a random subspace in dim space
    X = np.random.randn(dim, d)
    D = X.dot(X.T)
    U, _, _ = np.linalg.svd(D)
    U = U[:, 0:d]
    print(np.sum(U[:, 0]*U[:, 1])) # These two columns are orthogonal
    Y = U.dot(R)
    return Y


def rectangle(l = 1, w = 1, pts = 50, center = (0,0), fill = False, fill_pts = 50):
    """
    Creates a rectangle in 2-D spae.

    Parameters:
    l : int
        The distance of the length (y-axis).
    w : int
        The distance of the width (x-axis).
    center : tuple
        The x,y coordinate for the center of the rectangle
        - center[0] : int
            x coordinate for the center of the rectangle
        - center[1] : int
            y coordinate for the center of the rectangle
    pts : int
        The number of points to use to create the border of the shape
    fill : boolean
        Initally False, determines if you want to consider points inside the shape
    fill_pts : int
        Only used when fill is True; defines the number of points that are filled in the shape
    Returns:
    ret : np.array(2, pts)
        Returns the array that is used to make the rectangle in 2 dimensions.
    """
    if pts%4 != 0:
        pts += -pts%4
        print("Constructing rectangle with {} points".format(pts))
    retX = np.zeros(pts)
    retY = np.zeros(pts)
    centerX = center[0]
    startX = centerX-(w/2)
    side1X = np.linspace(startX, startX+w, pts//4)
    side2X = np.linspace(startX+w, startX, pts//4)
    retX[0:pts//4] = side1X
    retX[pts//4:(2*pts)//4] = startX+w
    retX[(2*pts)//4:(3*pts)//4] = side2X
    retX[(3*pts)//4::] = startX
    
    centerY = center[1]
    startY = centerY-(l/2)
    side1Y = np.linspace(startY, startY+l, pts//4)
    side2Y = np.linspace(startY+l, startY, pts//4)
    retY[0:pts//4] = startY
    retY[pts//4:(2*pts)//4] = side1Y
    retY[(2*pts)//4:(3*pts)//4] = startY+l
    retY[(3*pts)//4::] = side2Y
    if fill:
        i = 0
        while i < fill_pts:
            min_x = startX
            max_x = startX + w
            min_y = startY
            max_y = startY + l
            ptX = np.random.uniform(min_x, max_x)
            ptY = np.random.uniform(min_y, max_y)
            retX = np.append(retX, ptX)
            retY = np.append(retY, ptY)
            i += 1
    ret = []
    ret.append(retX)
    ret.append(retY)
    ret = np.array(ret)
    return ret
def ellipse(xr = 1, yr = 1, pts = 50, center=(0,0), fill = False, fill_pts = 50):
    """
    Creates an ellipse in 2-D spae.

    Parameters:
    xr : int
        The distance of the radius (x-axis).
    yr : int
        The distance of the radius (y-axis).
    center : tuple
        The x,y coordinate for the center of the rectangle
        - center[0] : int
            x coordinate for the center of the rectangle
        - center[1] : int
            y coordinate for the center of the rectangle
    pts : int
        The number of points to use to create the border of the shape
    fill : boolean
        Initally False, determines if you want to consider points inside the shape
    fill_pts : int
        Only used when fill is True; defines the number of points that are filled in the shape
        
    Returns:
    ret : np.array(2, pts)
        Returns the array that is used to make the ellipse in 2 dimensions.
    """
    ellX = np.zeros(pts)
    ellY = np.zeros(pts)

    theta = np.linspace(0, 2*np.pi, pts)

    x = (center[0] + xr)*np.cos(theta)
    y = (center[1] + yr)*np.sin(theta)

    ellX = x
    ellY = y

    if fill:
        i = 0
        while i < fill_pts:
            x_min = center[0] - xr
            x_max = center[0] + xr
            y_min = center[1] - yr
            y_max = center[1] + yr
            ptX = np.random.uniform(x_min, x_max)
            ptY = np.random.uniform(y_min, y_max)
            if ((ptX - center[0])**2 / xr**2) + ((ptY - center[1])**2 / yr**2) <= 1:
                ellX = np.append(ellX, ptX)
                ellY = np.append(ellY, ptY)
                i += 1
    
    ell = []
    ell.append(ellX)
    ell.append(ellY)
    ell = np.array(ell)
    return ell
def eq_triangle(r = 1, pts = 50, center = (0,0)):
    """
    Creates a rectangle in 2-D spae.

    Parameters:
    r : int
        The distance of the radius, which is used to calculate starting position, and side length
    center : tuple
        The x,y coordinate for the center of the rectangle
        - center[0] : int
            x coordinate for the center of the rectangle
        - center[1] : int
            y coordinate for the center of the rectangle
    pts : int
        The number of points to use to create the shape
        
    Returns:
    ret : np.array(2, pts)
        Returns the array that is used to make the triangle in 2 dimensions.
    """
    if pts%3 != 0:
        pts += -pts%3
        print("Constructing triangle with {} points".format(pts))
    tri = np.zeros((2, pts))   
    h_c = r/2
    s = 2 * h_c * np.sqrt(3)
    startX = center[0] - (s/2)
    startY = center[1] - h_c
    side1X = np.linspace(startX, startX+s, pts//3)
    side2X = np.linspace(startX+s, startX+(s/2), pts//3)
    side3X = np.linspace(startX+(s/2), startX, pts//3)
    tri[0][0:pts//3] = side1X
    tri[0][pts//3:(2*pts)//3] = side2X
    tri[0][(2*pts)//3::] = side3X
    
    side1Y = np.linspace(startY, startY, pts//3)
    side2Y = np.linspace(startY, startY+s, pts//3)
    side3Y = np.linspace(startY+s, startY, pts//3)
    tri[1][0:pts//3] = side1Y
    tri[1][pts//3:(2*pts)//3] = side2Y
    tri[1][(2*pts)//3::] = side3Y
    return tri

def reg_polygon(s = 3, r = 1, pts = 50, center = (0,0), fill = False, fill_pts = 50):
    """
    s : int
        The number of sides the polygon has
    r : int
        The length of the radius
    pts : int
        The number of points to use to create the border of the shape
    center : (int, int)
        The X-Y coordinate of the center of the polygon
    fill : boolean
        Initally False, determines if you want to consider points inside the shape
    fill_pts : int
        Only used when fill is True; defines the number of points that are filled in the shape

    Returns : 
    poly : np.array(2, pts ( + fill_pts?))
        Returns a 2-D array of the polygon given, where the number of points depends on the setting for fill
    """
    if pts%s != 0:
        pts += -pts%s
        print("Constructing polygon of {} sides with {} points".format(s, pts))   
    a = (s-2)*np.pi/2
    side = 2 * r * np.cos(a)
    h = r * np.sin(a)



    polyX = np.zeros(pts)
    polyY = np.zeros(pts)
    verticesX = []
    verticesY = []
    if s%2 != 0:
        startX = center[0] - (side/2)
        startY = center[1]
        angles = np.linspace(np.pi/2, 5*np.pi/2, s+1)
    else:    
        startX = center[0]
        startY = center[1] - h
        angles = np.linspace(np.pi/s+np.pi/2, (2*s+1)*np.pi/s+np.pi/2, s+1)
    for i in range(s):
        sideX = np.linspace(startX + r*np.cos(angles[i]), startX + r*np.cos(angles[i+1]), pts//s, endpoint=False)
        polyX[i*pts//s: (i+1)*pts//s] = sideX
        verticesX.append(sideX[0])

        sideY = np.linspace(startY + r*np.sin(angles[i]), startY + r*np.sin(angles[i+1]), pts//s, endpoint=False)
        polyY[i*pts//s: (i+1)*pts//s] = sideY
        verticesY.append(sideY[0])

    if fill:
        vertices = []
        vertices.append(verticesX)
        vertices.append(verticesY)
        i = 0
        while i < fill_pts:
            inside = False
            ptX = np.random.uniform(min(verticesX), max(verticesX))
            ptY = np.random.uniform(min(verticesY), max(verticesY))

            p1X = verticesX[0]
            p1Y = verticesY[0]
            
            for j in range(1, len(verticesX)+1):
                p2X = verticesX[j%len(verticesX)]
                p2Y = verticesY[j%len(verticesY)]
                if ptY > min(p1Y, p2Y):
                    if ptY <= max(p1Y, p2Y):
                        if ptX <= max(p1X, p2X):
                            x_int = (ptY-p1Y) * (p2X-p1X) / (p2Y-p1Y)+p1X
                            if p1X == p2X or ptX <= x_int:
                                inside = not inside
                p1X = p2X
                p1Y = p2Y
            
            if inside:
                polyX = np.append(polyX, (ptX))
                polyY = np.append(polyY, (ptY))
                i += 1
    poly = []
    poly.append(polyX)
    poly.append(polyY)
    return poly

In [ ]:
def makeBunchORectangles():
    for _ in range(2):
        fill = bool(_)
        for num in range(50):
            l = random.uniform(0.0001, 2.9999)
            w = random.uniform(0.0001, 2.9999)
            pts = random.randint(25, 250)
            center = (random.uniform(-3, 3), random.uniform(-3, 3))
            fill_pts = random.randint(25, 250)

            arr = np.array([create_shape(rectangle(l=l, w=w, pts=pts, center=center, fill=fill, fill_pts=fill_pts), 128, 2)]).astype(np.float32)
            z = torch.from_numpy(arr)
            maestro_ones_ts = "/opt/research/AudioSteno/datasets/maestrowithones/maestroWithOnes.ts"
            model = torch.jit.load(maestro_ones_ts).eval()
            sr = 44100
            print(arr.shape)
            x = model.decode(z).detach().numpy().reshape(-1)
            display(Audio(x, rate=sr))

            sf.write('/opt/research/AudioSteno/datasets/maestro/generatedAudio/{}{}{}.wav'.format("128rectangle_", fill, num+1), x, sr)

makeBunchORectangles()

In [4]:
def makeBunchOEllipses():
    for _ in range(2):
        fill = bool(_)
        for num in range(50):
            xr = random.uniform(0.0001, 2.9999)
            yr = random.uniform(0.0001, 2.9999)
            pts = random.randint(25, 250)
            center = (random.uniform(-3, 3), random.uniform(-3, 3))
            fill_pts = random.randint(25, 250)

            arr = np.array([create_shape(ellipse(xr, yr, pts=pts, center=center, fill=fill, fill_pts=fill_pts), 128, 2)]).astype(np.float32)
            z = torch.from_numpy(arr)
            maestro_ones_ts = "/opt/research/AudioSteno/datasets/maestro/train/maestro_train_original_e18d54798e/version_0/checkpoints/maestro_train_original_e18d54798e.ts"
            model = torch.jit.load(maestro_ones_ts).eval()
            sr = 44100
            print(arr.shape)
            x = model.decode(z).detach().numpy().reshape(-1)
            #display(Audio(x, rate=sr))

            sf.write('/opt/research/AudioSteno/datasets/maestro/generatedAudio/{}{}{}.wav'.format("128ellipse_", fill, num+1), x, sr)

makeBunchOEllipses()

7.632783294297951e-17
(1, 128, 228)
-2.498001805406602e-16
(1, 128, 231)
2.8796409701215e-16
(1, 128, 157)
3.469446951953614e-18
(1, 128, 212)
-2.220446049250313e-16
(1, 128, 84)
2.0122792321330962e-16
(1, 128, 106)
1.1102230246251565e-16
(1, 128, 169)
-4.163336342344337e-17
(1, 128, 229)
-9.020562075079397e-17
(1, 128, 97)
1.249000902703301e-16
(1, 128, 78)
3.469446951953614e-17
(1, 128, 181)
-2.1510571102112408e-16
(1, 128, 178)
-2.7755575615628914e-17
(1, 128, 103)
-2.7755575615628914e-17
(1, 128, 236)
3.469446951953614e-17
(1, 128, 219)
-3.0878077872387166e-16
(1, 128, 195)
-1.682681771697503e-16
(1, 128, 149)
-1.3877787807814457e-17
(1, 128, 248)
2.0816681711721685e-17
(1, 128, 70)
-3.8163916471489756e-17
(1, 128, 97)
2.5673907444456745e-16
(1, 128, 123)
-2.7582103268031233e-16
(1, 128, 125)
-1.1796119636642288e-16
(1, 128, 212)
-9.020562075079397e-17
(1, 128, 61)
1.97758476261356e-16
(1, 128, 70)
-1.3877787807814457e-17
(1, 128, 173)
3.2959746043559335e-17
(1, 128, 184)
-4.163336

In [5]:
def makeBunchORegPolys():
    for _ in range(2):
        fill = bool(_)
        for num in range(50):
            s = random.randint(3, 200)
            r = random.uniform(0.0001, 2.9999)
            pts = random.randint(25, 250)
            center = (random.uniform(-3, 3), random.uniform(-3, 3))
            fill_pts = random.randint(25, 250)

            arr = np.array([create_shape(reg_polygon(s, r, pts=pts, center=center, fill=fill, fill_pts=fill_pts), 128, 2)]).astype(np.float32)
            z = torch.from_numpy(arr)
            maestro_ones_ts = "/opt/research/AudioSteno/datasets/maestro/train/maestro_train_original_e18d54798e/version_0/checkpoints/maestro_train_original_e18d54798e.ts"
            model = torch.jit.load(maestro_ones_ts).eval()
            sr = 44100
            print(arr.shape)
            x = model.decode(z).detach().numpy().reshape(-1)
            #display(Audio(x, rate=sr))

            sf.write('/opt/research/AudioSteno/datasets/maestro/generatedAudio/{}{}{}.wav'.format("128reg_poly_", fill, num+1), x, sr)

makeBunchORegPolys()

Constructing polygon of 151 sides with 302 points
-1.0581813203458523e-16
(1, 128, 302)
Constructing polygon of 157 sides with 157 points
-1.8735013540549517e-16
(1, 128, 157)
Constructing polygon of 5 sides with 235 points
-7.632783294297951e-17
(1, 128, 235)
Constructing polygon of 166 sides with 166 points
2.983724378680108e-16
(1, 128, 166)
Constructing polygon of 13 sides with 182 points
9.71445146547012e-17
(1, 128, 182)
Constructing polygon of 122 sides with 244 points
-2.0816681711721685e-17
(1, 128, 244)
Constructing polygon of 52 sides with 208 points
-6.938893903907228e-18
(1, 128, 208)
Constructing polygon of 40 sides with 240 points
-9.367506770274758e-17
(1, 128, 240)
Constructing polygon of 83 sides with 166 points
-4.163336342344337e-17
(1, 128, 166)
Constructing polygon of 111 sides with 222 points
2.0816681711721685e-17
(1, 128, 222)
Constructing polygon of 8 sides with 72 points
-1.734723475976807e-16
(1, 128, 72)
Constructing polygon of 110 sides with 110 points
3.4

In [34]:
maestro_ones_ts = "/opt/research/datasets/maestrowithones/maestroWithOnes.ts"
maestro_orig_ts = "/opt/research/datasets/maestro/train/maestro_train_original_e18d54798e/version_0/checkpoints/maestro_train_original_e18d54798e.ts"

model = torch.jit.load(maestro_orig_ts).eval()

sr = 44100

In [42]:
shape = 'rectangle'
arr = np.array([rectangle(0.75, 0.5, [0.5,0.5], 225).astype(np.float32)])

Constructing rectangle with 228 points


In [44]:
shape = "ellipse"
arr = np.array([ellipse(0.3, 0.4, [0.5,0.5], 225).astype(np.float32)])

In [46]:
shape = "eq_triangle"
arr = np.array([eq_triangle(0.45, 0.23, [0.5,0.5], 255).astype(np.float32)])

In [47]:
z = torch.from_numpy(arr)
print(arr.shape)
x = model.decode(z).detach().numpy().reshape(-1)

display(Audio(x, rate=sr))
sf.write('/opt/research/datasets/maestro/generatedAudio/{}.wav'.format(shape), x, sr)

(1, 2, 255)
